In [ ]:
!pip install datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd

pd.options.display.max_columns= None

In [ ]:
dataset = load_dataset("AnikaBasu/CyberbullyingDataset")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
df = pd.DataFrame(dataset['train'])

In [ ]:
df.head()

,input,text,output,instruction
0,,Here is a classification that determines wheth...,not_cyberbullying,"In other words #katandandre, your food was cra..."
1,,Here is a classification that determines wheth...,not_cyberbullying,"In other words #katandandre, your food was cra..."
2,,Here is a classification that determines wheth...,not_cyberbullying,Why is #aussietv so white? #MKR #theblock #ImA...
3,,Here is a classification that determines wheth...,not_cyberbullying,@XochitlSuckkks a classy whore? Or more red ve...
4,,Here is a classification that determines wheth...,not_cyberbullying,"@Jason_Gio meh. :P thanks for the heads up, b..."


In [ ]:
def drop_columns(df, columns):
    return df.drop(columns=columns, inplace=False)

columns_to_drop = ['input', 'text']
df = drop_columns(df, columns_to_drop)

In [ ]:
df['output'].value_counts()

output
gender                 612
religion               562
other_cyberbullying    483
not_cyberbullying      434
age                    433
ethnicity              432
Name: count, dtype: int64

In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Assign values to cyberbullying types
cyberbullying_type_mapping = {
    "age": 1,
    "ethnicity": 2,
    "gender": 3,
    "not_cyberbullying": 4,
    "other_cyberbullying": 5,
    "religion": 6
}
df['output'] = df['output'].map(cyberbullying_type_mapping)

In [ ]:

def preprocess_text(text):

    # Lowercasing
    text = text.lower()

    # Replace links with 'link'
    text = re.sub(r'http\S+|www\S+', 'link', text)

    # Remove mentions (words starting with @)
    text = re.sub(r'@\w+', '', text)

    #remove hashtags
    text = re.sub(r'#\w+', '', text)

    # Tokenization
    tokens = word_tokenize(text)

    # Removing Punctuation and Special Characters
    tokens = [word for word in tokens if word.isalnum()]

    # Removing Stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into a single string
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text

# Applying text preprocessing to the 'instruction' column in the DataFrame
df['instruction'] = df['instruction'].apply(preprocess_text)

print(df.head())

   output                                        instruction
0       4                            words food crapilicious
1       4                            words food crapilicious
2       4                                              white
3       4                   classy whore red velvet cupcakes
4       4  meh p thanks heads concerned another angry dud...


In [ ]:

#Tokenize data
from nltk.tokenize import word_tokenize

df['tokens'] = df['instruction'].apply(word_tokenize)

# Display the DataFrame with tokenized text
print(df[['instruction', 'tokens']].head())

                                         instruction  \
0                            words food crapilicious   
1                            words food crapilicious   
2                                              white   
3                   classy whore red velvet cupcakes   
4  meh p thanks heads concerned another angry dud...   

                                              tokens  
0                        [words, food, crapilicious]  
1                        [words, food, crapilicious]  
2                                            [white]  
3             [classy, whore, red, velvet, cupcakes]  
4  [meh, p, thanks, heads, concerned, another, an...  


In [ ]:
# NLTK stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Function to remove stopwords
def remove_stopwords(text):
    return ' '.join(word for word in text.split() if word.lower() not in stop_words)

df['instruction'] = df['instruction'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['instruction']

0                                 words food crapilicious
1                                 words food crapilicious
2                                                   white
3                        classy whore red velvet cupcakes
4       meh p thanks heads concerned another angry dud...
                              ...                        
2951    u tell ppl ok someone ridiculed ur race cuz so...
2952    right battle dumb fuck cuz whites see nigger e...
2953                    mad rt fuck obama dumb ass nigger
2954    fuck dumb nigger would never stand fuckin show...
2955    pussy ass bitch dumb muthafuker like fuck u ni...
Name: instruction, Length: 2956, dtype: object

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit the vectorizer to your data
tfidf_matrix = tfidf_vectorizer.fit_transform(df['instruction'])
tfidf_features = tfidf_matrix.toarray()

print(tfidf_features)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_features, df['output'], test_size=0.2, random_state=42)


In [ ]:
# Train the SVC model
svc_model = SVC(kernel='linear')
svc_model.fit(X_train, y_train)

# Predictting outputs for the training data
y_train_pred = svc_model.predict(X_train)

# Calculate metrics for the training data
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_f1_score = f1_score(y_train, y_train_pred, average='weighted')

print("Training Metrics:")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1-Score:", train_f1_score)

Training Metrics:
Accuracy: 0.9758883248730964
Precision: 0.9764320484424392
Recall: 0.9758883248730964
F1-Score: 0.9758465951565614


In [ ]:
# Predict outputs for the test data
y_test_pred = svc_model.predict(X_test)

# Calculate metrics for the test data
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted')
test_recall = recall_score(y_test, y_test_pred, average='weighted')
test_f1_score = f1_score(y_test, y_test_pred, average='weighted')

print("\nTest Metrics:")
print("Accuracy:", test_accuracy)
print("Precision:", test_precision)
print("Recall:", test_recall)
print("F1-Score:", test_f1_score)



Test Metrics:
Accuracy: 0.793918918918919
Precision: 0.8120842796933005
Recall: 0.793918918918919
F1-Score: 0.7952599358660396


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Define and train the linear regression model
linear_regression = LinearRegression()
linear_regression.fit(X_train, y_train)

# Predict probabilities for the training set
y_train_pred_prob = linear_regression.predict(X_train)

# Define a threshold for binarizing the predictions
threshold = 0.5

# Binarize the predictions based on the threshold
y_train_pred = (y_train_pred_prob > threshold).astype(int)

# Evaluate train set metrics
train_accuracy = accuracy_score(y_train, y_train_pred)
train_precision = precision_score(y_train, y_train_pred, average='weighted')
train_recall = recall_score(y_train, y_train_pred, average='weighted')
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

# Print train set evaluation metrics
print("Train Set Metrics:")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1-Score:", train_f1)

Train Set Metrics:
Accuracy: 0.1484771573604061
Precision: 0.022045466257826794
Recall: 0.1484771573604061
F1-Score: 0.03839077880920998


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#TEST PREDICTION OF LINEAR REG MODEL

# Predict on the test set
y_test_pred_prob = linear_regression.predict(X_test)
y_test_pred = (y_test_pred_prob > threshold).astype(int)

# Evaluate test set metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_precision = precision_score(y_test, y_test_pred, average='weighted')
test_recall = recall_score(y_test, y_test_pred, average='weighted')
test_f1 = f1_score(y_test, y_test_pred, average='weighted')

# Print test set evaluation metrics
print("\nTest Set Metrics:")
print("Accuracy:", test_accuracy)
print("Precision:", test_precision)
print("Recall:", test_recall)
print("F1-Score:", test_f1)


Test Set Metrics:
Accuracy: 0.07432432432432433
Precision: 0.020383259513694296
Recall: 0.07432432432432433
F1-Score: 0.03199262254380365


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


In [ ]:
import numpy as np

unique_labels = np.unique(y_train)
print("Unique Labels:", unique_labels)

Unique Labels: [1 2 3 4 5 6]


In [ ]:
# Encode the target variable
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # No need to subtract 1 here
y_test_encoded = label_encoder.transform(y_test)  # No need to subtract 1 here

# Define a neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.5),
    Dense(6, activation='sigmoid')  # 6 output neurons for 6 categories
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',  # Use sparse categorical crossentropy
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_encoded, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Predict probabilities for the training set using the neural network model
y_train_pred_prob = model.predict(X_train)
threshold = 0.5  # Define the threshold for binarizing the predictions
y_train_pred = (y_train_pred_prob > threshold).astype(int)

# Convert predicted probabilities to class labels
y_train_pred_labels = np.argmax(y_train_pred, axis=1)

# Evaluate train set metrics
train_accuracy = accuracy_score(y_train_encoded, y_train_pred_labels)
train_precision = precision_score(y_train_encoded, y_train_pred_labels, average='weighted')
train_recall = recall_score(y_train_encoded, y_train_pred_labels, average='weighted')
train_f1 = f1_score(y_train_encoded, y_train_pred_labels, average='weighted')

# Print train set evaluation metrics
print("Train Set Metrics:")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1-Score:", train_f1)

Epoch 1/20
60/60 [==============================] - 2s 14ms/step - loss: 1.7803 - accuracy: 0.2152 - val_loss: 1.7495 - val_accuracy: 0.3679
Epoch 2/20
60/60 [==============================] - 1s 9ms/step - loss: 1.6784 - accuracy: 0.3961 - val_loss: 1.5572 - val_accuracy: 0.6681
Epoch 3/20
60/60 [==============================] - 1s 13ms/step - loss: 1.3906 - accuracy: 0.5769 - val_loss: 1.1617 - val_accuracy: 0.7463
Epoch 4/20
60/60 [==============================] - 1s 13ms/step - loss: 1.0093 - accuracy: 0.6965 - val_loss: 0.8573 - val_accuracy: 0.7674
Epoch 5/20
60/60 [==============================] - 1s 14ms/step - loss: 0.7131 - accuracy: 0.7673 - val_loss: 0.7056 - val_accuracy: 0.7780
Epoch 6/20
60/60 [==============================] - 1s 12ms/step - loss: 0.5735 - accuracy: 0.8033 - val_loss: 0.6287 - val_accuracy: 0.7970
Epoch 7/20
60/60 [==============================] - 1s 12ms/step - loss: 0.5070 - accuracy: 0.8213 - val_loss: 0.5866 - val_accuracy: 0.7886
Epoch 8/20
60/

In [ ]:
# Predict probabilities for the test set using the neural network model
y_test_pred = model.predict(X_test)

# Convert predicted probabilities to class labels
y_test_pred_labels = np.argmax(y_test_pred, axis=1)

# Evaluate test set metrics
test_accuracy = accuracy_score(y_test_encoded, y_test_pred_labels)
test_precision = precision_score(y_test_encoded, y_test_pred_labels, average='weighted')
test_recall = recall_score(y_test_encoded, y_test_pred_labels, average='weighted')
test_f1 = f1_score(y_test_encoded, y_test_pred_labels, average='weighted')

# Print test set evaluation metrics
print("Test Set Metrics:")
print("Accuracy:", test_accuracy)
print("Precision:", test_precision)
print("Recall:", test_recall)
print("F1-Score:", test_f1)

19/19 [==============================] - 0s 3ms/step
Test Set Metrics:
Accuracy: 0.785472972972973
Precision: 0.7801876601417694
Recall: 0.785472972972973
F1-Score: 0.7821415499485611


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [ ]:
#LSTN MODEL


# Convert X_train and X_test to lists of strings
X_train_texts = [str(text) for text in X_train]
X_test_texts = [str(text) for text in X_test]

# Tokenization and Padding for LSTM
maxlen = 100  # Maximum sequence length
max_words = 10000  # Maximum number of words to consider as features

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train_texts)

# Tokenization and Padding for LSTM on train dataset
X_train_lstm = tokenizer.texts_to_sequences(X_train_texts)
X_train_lstm = pad_sequences(X_train_lstm, maxlen=maxlen)

# Tokenization and Padding for LSTM on test dataset
X_test_lstm = tokenizer.texts_to_sequences(X_test_texts)
X_test_lstm = pad_sequences(X_test_lstm, maxlen=maxlen)

# Define LSTM model architecture
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 32, input_length=maxlen))
lstm_model.add(LSTM(32))
lstm_model.add(Dense(1, activation='sigmoid'))

# Compile and train LSTM model
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_lstm, y_train, epochs=10, batch_size=32)

# Make predictions on train set
lstm_train_preds = (lstm_model.predict(X_train_lstm) > 0.5).astype(int)

# Calculate evaluation metrics for train set
train_accuracy = accuracy_score(y_train, lstm_train_preds)
train_precision = precision_score(y_train, lstm_train_preds, average='weighted')
train_recall = recall_score(y_train, lstm_train_preds, average='weighted')
train_f1 = f1_score(y_train, lstm_train_preds, average='weighted')

# Print evaluation metrics for train set
print("Train Set Metrics:")
print("Accuracy:", train_accuracy)
print("Precision:", train_precision)
print("Recall:", train_recall)
print("F1-Score:", train_f1)

Epoch 1/10
74/74 [==============================] - 6s 47ms/step - loss: -9.2310 - accuracy: 0.1459
Epoch 2/10
74/74 [==============================] - 3s 46ms/step - loss: -25.8391 - accuracy: 0.1485
Epoch 3/10
74/74 [==============================] - 3s 40ms/step - loss: -34.5346 - accuracy: 0.1485
Epoch 4/10
74/74 [==============================] - 3s 39ms/step - loss: -41.9639 - accuracy: 0.1485
Epoch 5/10
74/74 [==============================] - 3s 46ms/step - loss: -48.9948 - accuracy: 0.1485
Epoch 6/10
74/74 [==============================] - 4s 47ms/step - loss: -55.8446 - accuracy: 0.1485
Epoch 7/10
74/74 [==============================] - 3s 40ms/step - loss: -62.5722 - accuracy: 0.1485
Epoch 8/10
74/74 [==============================] - 3s 40ms/step - loss: -69.2268 - accuracy: 0.1485
Epoch 9/10
74/74 [==============================] - 4s 47ms/step - loss: -75.8227 - accuracy: 0.1485
Epoch 10/10
74/74 [==============================] - 2s 14ms/step
Train Set Metrics:
Accurac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
